# TranSTR CausalVid - W&B Version (CLIP Features)

Notebook hoàn chỉnh cho **training và evaluation** model TranSTR trên CausalVidQA.

**Tính năng:**
- **CLIP ViT-Large-Patch14 features** (D=1024) thay vì ViT features (768)
- DeBERTa encode text **real-time** (không cần pre-extraction)
- **Weights & Biases (W&B)** để log metrics và checkpoints
- Detailed evaluation theo paper metrics (D, E, PAR, CAR, Acc(ALL))

In [ ]:
# CELL 1: Git Clone & Setup
import os
REPO_URL = "https://github.com/DanielQH07/tranSTR_Casual.git" 
REPO_NAME = "tranSTR_Casual"
BRANCH = "origin" 

if not os.path.exists(REPO_NAME):
    print(f"Cloning {REPO_URL}...")
    !git clone {REPO_URL} -b {BRANCH}
else:
    print("Repo already exists.")

# Change Directory to the repo root 
if os.path.basename(os.getcwd()) != REPO_NAME:
    try:
        target_dir = os.path.join(os.getcwd(), REPO_NAME, "causalvid")
        if os.path.exists(target_dir):
             os.chdir(target_dir)
        elif os.path.exists(REPO_NAME):
             os.chdir(REPO_NAME)
        
        print(f"Changed directory to: {os.getcwd()}")
    except Exception as e:
             print(f"Could not set working directory: {e}")

In [ ]:
# CELL 2: Install & Login W&B (với API Key trực tiếp)
print('=== CELL 2: W&B Setup ===')
!pip install -q wandb

import wandb

# ============================================
# WANDB CONFIG - THAY THẾ BẰNG API KEY CỦA BẠN
# ============================================
WANDB_API_KEY = 'YOUR_WANDB_API_KEY_HERE'  # 🔴 Paste your W&B API key here
WANDB_PROJECT = 'transtr-causalvid-clip'   # Project name on W&B
WANDB_ENTITY = None                        # Your W&B username/team (None = default)

# Login with API key (no interactive prompt)
wandb.login(key=WANDB_API_KEY)
print('✅ W&B logged in successfully!')

In [ ]:
# CELL 3: Imports
print('=== CELL 3: Imports ===')
import os, torch, numpy as np, pandas as pd, json
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from utils.util import set_seed, set_gpu_devices
from DataLoader import VideoQADataset
from networks.model import VideoQAmodel
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm
print('Imports OK')

In [ ]:
# CELL 4: Train/Eval functions (với W&B logging)
print('=== CELL 4: Functions ===')

def train_epoch(model, optimizer, loader, xe, device, epoch):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    pbar = tqdm(loader, desc=f'Epoch {epoch}', leave=False)
    for batch_idx, batch in enumerate(pbar):
        ff, of, q, a, ans_id, _ = batch
        ff, of, tgt = ff.to(device), of.to(device), ans_id.to(device)
        
        out = model(ff, of, q, a)
        loss = xe(out, tgt)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
        correct += (out.argmax(-1) == tgt).sum().item()
        total += tgt.size(0)
        
        # Update progress bar
        pbar.set_postfix({'loss': total_loss/(batch_idx+1), 'acc': correct/total*100})
        
        # Log batch metrics to W&B (every 50 batches)
        if batch_idx % 50 == 0:
            wandb.log({
                'batch_loss': loss.item(),
                'batch_acc': (out.argmax(-1) == tgt).float().mean().item() * 100,
                'batch': epoch * len(loader) + batch_idx
            })
    
    return total_loss / len(loader), correct / total * 100

def eval_epoch(model, loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in loader:
            ff, of, q, a, ans_id, _ = batch
            out = model(ff.to(device), of.to(device), q, a)
            correct += (out.argmax(-1) == ans_id.to(device)).sum().item()
            total += ans_id.size(0)
    return correct / total * 100

print('Functions defined with W&B logging!')

In [ ]:
# Merge CLIP Features (train/test/valid -> merged folder)
import os
import shutil
from tqdm.auto import tqdm

# ============================================
# 🔴 UPDATE THESE PATHS
# ============================================
CLIP_SPLIT_PATH = '/kaggle/input/clip-vit-large-patch14-features'  # Folder chứa train/test/valid
CLIP_MERGED_PATH = '/kaggle/working/clip_features_merged'          # Folder output đã gộp

# Check structure
print(f'\nSource: {CLIP_SPLIT_PATH}')
if os.path.exists(CLIP_SPLIT_PATH):
    subfolders = os.listdir(CLIP_SPLIT_PATH)
    print(f'Subfolders found: {subfolders}')
else:
    print('❌ Source path not found!')
    subfolders = []

# Merge folders
os.makedirs(CLIP_MERGED_PATH, exist_ok=True)

total_copied = 0
for split in ['train', 'test', 'valid', 'val']:  # Try common folder names
    split_folder = os.path.join(CLIP_SPLIT_PATH, split)
    if not os.path.exists(split_folder):
        continue
    
    pt_files = [f for f in os.listdir(split_folder) if f.endswith('.pt')]
    print(f'\n📁 {split}: {len(pt_files)} files')
    
    for fname in tqdm(pt_files, desc=f'Copying {split}'):
        src = os.path.join(split_folder, fname)
        dst = os.path.join(CLIP_MERGED_PATH, fname)
        
        # Skip if already exists
        if not os.path.exists(dst):
            shutil.copy2(src, dst)
            total_copied += 1

# Summary
final_count = len([f for f in os.listdir(CLIP_MERGED_PATH) if f.endswith('.pt')])
print(f'\n✅ Merge complete!')
print(f'   Total .pt files in merged folder: {final_count}')
print(f'   Merged path: {CLIP_MERGED_PATH}')

In [ ]:
# CELL 5: Setup Paths & Config
print('=== CELL 5: Paths & Config ===')

# ============================================
# KAGGLE INPUT PATHS - UPDATE THESE!
# ============================================
# 🔴 THAY ĐỔI: Sử dụng CLIP features (1024 dim) thay vì ViT features (768 dim)
CLIP_FEATURE_PATH = CLIP_MERGED_PATH  # 🔴 UPDATE PATH TO YOUR CLIP FEATURES
OBJ_FEATURE_PATH = '/kaggle/input/object-detection-causal-full'
ANNOTATION_PATH = '/kaggle/input/text-annotation/QA'
SPLIT_DIR = '/kaggle/input/casual-vid-data-split/split'

# Working directories
BASE = '/kaggle/working'
MODEL_DIR = os.path.join(BASE, 'models')
os.makedirs(MODEL_DIR, exist_ok=True)

# Verify paths
print('\n--- Path Verification ---')
def verify_path(name, path):
    if os.path.exists(path):
        items = os.listdir(path)[:3]
        print(f'✅ {name}: {items}')
        return True
    else:
        print(f'❌ {name}: NOT FOUND - {path}')
        return False

all_ok = True
all_ok &= verify_path('CLIP Features (1024D)', CLIP_FEATURE_PATH)
all_ok &= verify_path('Object Features', OBJ_FEATURE_PATH)
all_ok &= verify_path('Annotations', ANNOTATION_PATH)
all_ok &= verify_path('Splits', SPLIT_DIR)

if not all_ok:
    print('\n⚠️ Please update paths above!')

# ============================================
# CONFIG - CLIP ViT-Large-Patch14 (1024 dim)
# ============================================
RUN_TRAINING = True
MODEL_FILENAME = 'best_model_clip.ckpt'

# 🔴 Feature dimension for CLIP ViT-Large-Patch14
FEAT_DIM = 1024  # openai/clip-vit-large-patch14
print(f'\n🔧 Backbone: openai/clip-vit-large-patch14 (D={FEAT_DIM})')

class Config:
    # Paths - 🔴 CHANGED: Using CLIP features
    video_feature_root = CLIP_FEATURE_PATH
    object_feature_path = OBJ_FEATURE_PATH
    sample_list_path = ANNOTATION_PATH
    split_dir_txt = SPLIT_DIR
    
    # Model architecture
    topK_frame = 16
    objs = 20
    frames = 16
    select_frames = 5
    topK_obj = 12
    
    # 🔴 CHANGED: frame_feat_dim = 1024 (CLIP) instead of 768 (ViT)
    frame_feat_dim = FEAT_DIM  # 1024 for CLIP ViT-Large-Patch14
    
    obj_feat_dim = 2053
    d_model = 768
    word_dim = 768
    nheads = 8
    num_encoder_layers = 2
    num_decoder_layers = 2
    normalize_before = True
    activation = 'gelu'
    dropout = 0.3
    encoder_dropout = 0.3
    
    # Text encoder
    text_encoder_type = 'microsoft/deberta-base'
    freeze_text_encoder = False
    text_encoder_lr = 1e-5
    text_pool_mode = 1
    
    # Training
    bs = 8
    lr = 1e-5
    epoch = 20
    gpu = 0
    patience = 5
    gamma = 0.1
    decay = 1e-4
    n_query = 5
    
    # Other
    hard_eval = False
    pos_ratio = 1.0
    neg_ratio = 1.0
    a = 1.0
    num_workers = 4

args = Config()
set_gpu_devices(args.gpu)
set_seed(999)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'\nDevice: {device}')
print(f'Config loaded! frame_feat_dim = {args.frame_feat_dim}')

In [ ]:
# CELL 6: Create Datasets
print('=== CELL 6: Datasets ===')

MAX_TRAIN_SAMPLES = 2000  # Change to None for all samples

print('\n--- Creating TRAIN dataset ---')
train_ds = VideoQADataset(
    split='train', n_query=args.n_query, obj_num=args.objs,
    sample_list_path=args.sample_list_path,
    video_feature_path=args.video_feature_root,
    object_feature_path=args.object_feature_path,
    split_dir=args.split_dir_txt, topK_frame=args.topK_frame,
    max_samples=MAX_TRAIN_SAMPLES, verbose=True
)

print('\n--- Creating VAL dataset ---')
val_ds = VideoQADataset(
    split='val', n_query=args.n_query, obj_num=args.objs,
    sample_list_path=args.sample_list_path,
    video_feature_path=args.video_feature_root,
    object_feature_path=args.object_feature_path,
    split_dir=args.split_dir_txt, topK_frame=args.topK_frame,
    max_samples=None, verbose=True
)

print('\n--- Creating TEST dataset ---')
test_ds = VideoQADataset(
    split='test', n_query=args.n_query, obj_num=args.objs,
    sample_list_path=args.sample_list_path,
    video_feature_path=args.video_feature_root,
    object_feature_path=args.object_feature_path,
    split_dir=args.split_dir_txt, topK_frame=args.topK_frame,
    max_samples=None, verbose=True
)

train_loader = DataLoader(train_ds, args.bs, shuffle=True, num_workers=args.num_workers, pin_memory=True)
val_loader = DataLoader(val_ds, args.bs, shuffle=False, num_workers=args.num_workers, pin_memory=True)
test_loader = DataLoader(test_ds, args.bs, shuffle=False, num_workers=args.num_workers, pin_memory=True)

print('\n' + '='*60)
print('DATASET SUMMARY')
print('='*60)
print(f'Train: {len(train_ds)} samples -> {len(train_loader)} batches')
print(f'Val:   {len(val_ds)} samples -> {len(val_loader)} batches')
print(f'Test:  {len(test_ds)} samples -> {len(test_loader)} batches')
print(f'Feature dim: {args.frame_feat_dim} (CLIP ViT-Large-Patch14)')
print('='*60)

In [ ]:
# CELL 7: Model
print('=== CELL 7: Model ===')
cfg = {k: v for k, v in Config.__dict__.items() if not k.startswith('_')}
cfg['device'] = device
cfg['topK_frame'] = args.select_frames

print(f'Creating model with frame_feat_dim = {cfg["frame_feat_dim"]}')

model = VideoQAmodel(**cfg)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.decay)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=args.gamma, patience=args.patience)
xe = nn.CrossEntropyLoss()
save_path = os.path.join(MODEL_DIR, MODEL_FILENAME)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total params: {total_params/1e6:.1f}M')
print(f'Trainable:    {trainable_params/1e6:.1f}M')

In [ ]:
# CELL 8: Initialize W&B Run
print('=== CELL 8: Initialize W&B Run ===')

# Create W&B config dict
wandb_config = {
    'model': 'TranSTR',
    'backbone': 'openai/clip-vit-large-patch14',  # 🔴 CHANGED
    'frame_feat_dim': args.frame_feat_dim,         # 🔴 ADDED
    'text_encoder': args.text_encoder_type,
    'batch_size': args.bs,
    'learning_rate': args.lr,
    'epochs': args.epoch,
    'max_train_samples': MAX_TRAIN_SAMPLES,
    'd_model': args.d_model,
    'nheads': args.nheads,
    'num_encoder_layers': args.num_encoder_layers,
    'num_decoder_layers': args.num_decoder_layers,
    'dropout': args.dropout,
    'topK_frame': args.select_frames,
    'topK_obj': args.topK_obj,
    'train_samples': len(train_ds),
    'val_samples': len(val_ds),
    'test_samples': len(test_ds),
}

# Initialize W&B run
run = wandb.init(
    project=WANDB_PROJECT,
    entity=WANDB_ENTITY,
    config=wandb_config,
    name=f'transtr-clip1024-ep{args.epoch}-bs{args.bs}',  # 🔴 CHANGED name
    reinit=True
)

# Watch model for gradient logging
wandb.watch(model, log='gradients', log_freq=100)

print(f'✅ W&B Run initialized: {run.url}')

In [ ]:
# CELL 9: Training with W&B Logging
print('=== CELL 9: Training ===')

best_acc = 0
history = {'train_loss': [], 'train_acc': [], 'val_acc': []}

if RUN_TRAINING:
    for ep in range(1, args.epoch + 1):
        print(f'\nEpoch {ep}/{args.epoch}')
        
        # Train
        loss, train_acc = train_epoch(model, optimizer, train_loader, xe, device, ep)
        
        # Validate
        val_acc = eval_epoch(model, val_loader, device)
        scheduler.step(val_acc)
        
        # Log to history
        history['train_loss'].append(loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        # 📊 LOG TO W&B
        wandb.log({
            'epoch': ep,
            'train_loss': loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'learning_rate': optimizer.param_groups[0]['lr'],
            'best_val_acc': max(best_acc, val_acc)
        })
        
        print(f'Loss: {loss:.4f} | Train: {train_acc:.1f}% | Val: {val_acc:.1f}%')
        
        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print(f'✅ New best! Saved to {save_path}')
            
            # 📦 LOG CHECKPOINT TO W&B
            artifact = wandb.Artifact(
                name='best-model-clip',  # 🔴 CHANGED
                type='model',
                description=f'Best CLIP model at epoch {ep} with val_acc={val_acc:.2f}%',
                metadata={
                    'epoch': ep,
                    'val_acc': val_acc,
                    'train_acc': train_acc,
                    'train_loss': loss,
                    'backbone': 'clip-vit-large-patch14',
                    'frame_feat_dim': FEAT_DIM
                }
            )
            artifact.add_file(save_path)
            wandb.log_artifact(artifact)
            print(f'📤 Checkpoint uploaded to W&B!')
    
    print(f'\n🏆 Best Val Accuracy: {best_acc:.1f}%')
    
    # Log final summary
    wandb.run.summary['best_val_acc'] = best_acc
    wandb.run.summary['final_train_loss'] = history['train_loss'][-1]
    wandb.run.summary['total_epochs'] = args.epoch
    wandb.run.summary['backbone'] = 'clip-vit-large-patch14'
    wandb.run.summary['frame_feat_dim'] = FEAT_DIM
    
else:
    print('Skipping training (RUN_TRAINING=False)')

In [ ]:
# CELL 10: Detailed Evaluation with W&B Logging
print('=== CELL 10: Detailed Evaluation ===')
import seaborn as sns

def evaluate_detailed_v2(model, loader, device, log_to_wandb=True):
    """Tính metric trực tiếp từ batch với W&B logging."""
    model.eval()
    type_results = {}
    
    print("\n📊 Running Detailed Evaluation...")
    with torch.no_grad():
        for batch in tqdm(loader):
            ff, of, qns, ans_word, ans_id, qns_keys = batch
            ff, of = ff.to(device), of.to(device)
            
            out = model(ff, of, qns, ans_word)
            preds = out.argmax(dim=-1).cpu().numpy()
            targets = ans_id.numpy()
            
            for key, pred, target in zip(qns_keys, preds, targets):
                # Parse key to get video_id and type
                if key.endswith('_reason'):
                    if '_predictive_reason' in key:
                        idx = key.rfind('_predictive_reason')
                        video_id, qtype = key[:idx], 'predictive_reason'
                    elif '_counterfactual_reason' in key:
                        idx = key.rfind('_counterfactual_reason')
                        video_id, qtype = key[:idx], 'counterfactual_reason'
                    else:
                        parts = key.rsplit('_', 2)
                        video_id = parts[0] if len(parts) > 2 else key
                        qtype = '_'.join(parts[1:]) if len(parts) > 1 else 'unknown'
                else:
                    parts = key.rsplit('_', 1)
                    video_id, qtype = parts if len(parts) == 2 else (key, 'unknown')
                
                if qtype not in type_results:
                    type_results[qtype] = []
                type_results[qtype].append({
                    'video_id': video_id,
                    'pred': int(pred),
                    'target': int(target),
                    'correct': int(pred) == int(target)
                })
    
    # Calculate metrics
    metrics = {}
    metrics_map = {
        'Description': 'descriptive',
        'Explanation': 'explanatory',
        'Predictive-Answer': 'predictive',
        'Predictive-Reason': 'predictive_reason',
        'Counterfactual-Answer': 'counterfactual',
        'Counterfactual-Reason': 'counterfactual_reason'
    }
    
    print("\n" + "="*60)
    print("EVALUATION RESULTS")
    print("="*60)
    
    for name, qtype in metrics_map.items():
        if qtype in type_results:
            results = type_results[qtype]
            correct = sum(1 for r in results if r['correct'])
            total = len(results)
            acc = correct / total * 100 if total > 0 else 0
        else:
            correct, total, acc = 0, 0, 0
        metrics[name] = acc
        print(f"{name:<25} ==>   {acc:.2f}%  ({correct}/{total})")
    
    # Hard Metrics
    print("-" * 60)
    
    def calc_hard_metric(type_ans, type_reason, name):
        if type_ans not in type_results or type_reason not in type_results:
            metrics[name] = 0
            print(f"{name:<25} ==>   0.00%  (0/0 paired)")
            return
        
        ans_by_vid = {r['video_id']: r['correct'] for r in type_results[type_ans]}
        reason_by_vid = {r['video_id']: r['correct'] for r in type_results[type_reason]}
        common_vids = set(ans_by_vid.keys()) & set(reason_by_vid.keys())
        
        both_correct = sum(1 for vid in common_vids if ans_by_vid[vid] and reason_by_vid[vid])
        total = len(common_vids)
        acc = both_correct / total * 100 if total > 0 else 0
        metrics[name] = acc
        print(f"{name:<25} ==>   {acc:.2f}%  ({both_correct}/{total} paired)")
    
    calc_hard_metric('predictive', 'predictive_reason', 'PAR')
    calc_hard_metric('counterfactual', 'counterfactual_reason', 'CAR')
    
    print("-" * 60)
    
    # Acc (ALL)
    d_acc = metrics.get('Description', 0)
    e_acc = metrics.get('Explanation', 0)
    par_acc = metrics.get('PAR', 0)
    car_acc = metrics.get('CAR', 0)
    acc_all = (d_acc + e_acc + par_acc + car_acc) / 4
    metrics['Acc_ALL'] = acc_all
    print(f"{'Acc (ALL)':<25} ==>   {acc_all:.2f}%  ((D+E+PAR+CAR)/4)")
    print("="*60)
    
    # 📊 LOG FINAL METRICS TO W&B
    if log_to_wandb:
        wandb.log({
            'eval/Description': metrics['Description'],
            'eval/Explanation': metrics['Explanation'],
            'eval/Predictive_Answer': metrics['Predictive-Answer'],
            'eval/Predictive_Reason': metrics['Predictive-Reason'],
            'eval/Counterfactual_Answer': metrics['Counterfactual-Answer'],
            'eval/Counterfactual_Reason': metrics['Counterfactual-Reason'],
            'eval/PAR': metrics['PAR'],
            'eval/CAR': metrics['CAR'],
            'eval/Acc_ALL': acc_all
        })
        
        # Log summary
        wandb.run.summary['eval_Description'] = metrics['Description']
        wandb.run.summary['eval_Explanation'] = metrics['Explanation']
        wandb.run.summary['eval_PAR'] = metrics['PAR']
        wandb.run.summary['eval_CAR'] = metrics['CAR']
        wandb.run.summary['eval_Acc_ALL'] = acc_all
        print('📤 Metrics logged to W&B!')
    
    # Plot
    keys = ['Description', 'Explanation', 'PAR', 'CAR', 'Acc_ALL']
    values = [metrics.get(k, 0) for k in keys]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(keys, values, color=sns.color_palette("viridis", len(keys)))
    ax.set_ylim(0, 100)
    ax.set_ylabel('Accuracy (%)')
    ax.set_title('VideoQA Performance (CLIP ViT-Large-Patch14)')
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    for bar in bars:
        ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
                f'{bar.get_height():.1f}%', ha='center', va='bottom', fontweight='bold')
    plt.tight_layout()
    
    # Log chart to W&B
    if log_to_wandb:
        wandb.log({'eval_chart': wandb.Image(fig)})
    
    plt.savefig('eval_results_clip.png')
    plt.show()
    
    return metrics, type_results

# Run evaluation on VALIDATION set (has ground truth)
print("\n📌 Using VALIDATION SET for evaluation")
metrics, raw_results = evaluate_detailed_v2(model, val_loader, device, log_to_wandb=True)

In [ ]:
# CELL 11: Finish W&B Run
print('=== CELL 11: Finish W&B ===')

# Save metrics locally
with open('final_metrics_clip.json', 'w') as f:
    json.dump(metrics, f, indent=2)
print('Saved: final_metrics_clip.json')

# Log final artifact with all results
final_artifact = wandb.Artifact(
    name='final-results-clip',
    type='results',
    description='Final evaluation results and metrics (CLIP ViT-Large-Patch14)',
    metadata={
        'backbone': 'clip-vit-large-patch14',
        'frame_feat_dim': FEAT_DIM
    }
)
final_artifact.add_file('final_metrics_clip.json')
final_artifact.add_file('eval_results_clip.png')
if os.path.exists(save_path):
    final_artifact.add_file(save_path)
wandb.log_artifact(final_artifact)

# Finish run
wandb.finish()
print('\n✅ W&B run finished!')
print(f'View results at: https://wandb.ai/{WANDB_ENTITY or "your-username"}/{WANDB_PROJECT}')